In [3]:
%cd /local/musaeed/HLCV23

/local/musaeed/HLCV23


In [13]:
import os
import numpy as np
from tqdm.notebook import tqdm
from src.imagenet_labels import lab_dict
from lavis.processors.blip_processors import BlipCaptionProcessor
from transformers import ViTFeatureExtractor, ViTForImageClassification
import torch
from torchvision import transforms
from torch.utils.data import DataLoader
from PIL import Image  # Import the PIL library explicitly
import matplotlib.pyplot as plt
from src.dataloaders import imagenet_c_dataloader, imagenet_dataloader


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Transforms for the input data
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the image to (224, 224) to match the ViT input size
    transforms.ToTensor(),
    normalize,
])


# Function to load images using PIL explicitly
def pil_loader(path):
    with open(path, 'rb') as f:
        img = Image.open(f)
        return img.convert('RGB')

# Load the ImageNet dataset
data_dir = '/local/musaeed/HLCV23/data/imagenet'
dataset = datasets.ImageFolder(data_dir, transform=transform, loader=pil_loader)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

# Load the ViT model and feature extractor
model_name = "google/vit-base-patch16-224-in21k"  # ViT base model pre-trained on 21k classes
model_name = "google/vit-base-patch16-224"
feature_extractor = ViTFeatureExtractor.from_pretrained(model_name)
model = ViTForImageClassification.from_pretrained(model_name)
model = model.to(device)

cls_names = [lab_dict[i].replace('_', ' ') for i in os.listdir('/local/musaeed/HLCV23/data/imagenet')]

text_processor = BlipCaptionProcessor(prompt="A picture of ")
cls_prompt = [text_processor(cls_nm) for cls_nm in cls_names]
print(cls_prompt)

def get_acc(gt, preds):
    # The function calculates the accuracy of the predictions
    # gt - ground truth labels
    # preds - model predictions
    return ((preds.argmax(1) == gt).sum() / len(preds)).cpu().numpy()

def get_test_acc(model, loader, device='cuda'):
    model.eval()
    eval_acc = []
    with torch.no_grad():
        for batch in tqdm(loader, leave=False):
            ims, labels = batch
            ims, labels = ims.to(device), labels.to(device)
            
            outputs = model(ims)  # Directly use images

            val_acc = get_acc(labels.view(-1,), outputs.logits)
            eval_acc.append(val_acc)

    return np.mean(eval_acc)

print(get_test_acc(model, loader, device))



cuda


/local/musaeed/anaconda3/envs/t5/lib/python3.9/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
